In [2]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.7 MB/s eta 0:00:00


In [4]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 5.0 MB/s eta 0:00:00


In [5]:
import os
import io
import json
import time
import zipfile
import tempfile
import shutil
import traceback
from typing import List, Dict, Any, Tuple

import numpy as np
from sentence_transformers import SentenceTransformer
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

import gradio as gr
from pypdf import PdfReader

import google.generativeai as genai

REFS_DIR = "adgm_refs"
os.makedirs(REFS_DIR, exist_ok=True)

CHECKLIST = [
    "Articles of Association",
    "Memorandum of Association",
    "UBO Declaration Form",
    "Incorporation Application Form",
    "Register of Members and Directors",
]

DOCUMENT_KEYWORDS = {
    "Articles of Association": ["articles of association", "aoa"],
    "Memorandum of Association": ["memorandum of association", "moa"],
    "UBO Declaration Form": ["ubo declaration", "ubo", "ultimate beneficial owner"],
    "Incorporation Application Form": ["incorporation application", "application form", "incorporation form"],
    "Register of Members and Directors": ["register of members", "register of directors"],
}

JURISDICTION_KEYWORDS_WRONG = ["federal court", "federal courts", "u.a.e. federal", "dubai courts"]
SIGNATURE_KEYWORDS = ["signature", "signed", "for and on behalf", "authorised signatory"]
AMBIGUOUS_TERMS = ["may ", "should ", "endeavour to", "subject to the discretion", "best endeavours"]

EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

def extract_text_from_docx(path: str) -> str:
    try:
        doc = Document(path)
        paragraphs = [p.text for p in doc.paragraphs if p.text and p.text.strip()]
        return "\n".join(paragraphs)
    except Exception:
        return ""

def extract_text_from_pdf(path: str) -> str:
    try:
        reader = PdfReader(path)
        texts = []
        for p in reader.pages:
            try:
                texts.append(p.extract_text() or "")
            except Exception:
                continue
        return "\n".join(texts)
    except Exception:
        return ""

def chunk_text(text: str, chunk_size: int = 800, overlap: int = 100) -> List[str]:
    if not text:
        return []
    tokens = text.split()
    chunks = []
    i = 0
    while i < len(tokens):
        chunk = tokens[i:i+chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

class SimpleRAGIndex:
    def __init__(self, embed_model_name=EMBED_MODEL_NAME):
        self.model = SentenceTransformer(embed_model_name)
        self.records = []

    def add_text(self, text: str, source: str):
        if not text or not text.strip():
            return
        emb = self.model.encode(text, convert_to_numpy=True)
        self.records.append({"text": text, "source": source, "emb": emb})

    def build_from_folder(self, folder: str):
        self.records = []
        for fname in sorted(os.listdir(folder)):
            path = os.path.join(folder, fname)
            if fname.lower().endswith(".docx"):
                try:
                    if not zipfile.is_zipfile(path):
                        print("Skipping invalid DOCX reference:", fname)
                        continue
                except Exception:
                    continue
                text = extract_text_from_docx(path)
            elif fname.lower().endswith(".pdf"):
                text = extract_text_from_pdf(path)
            else:
                continue
            for chunk in chunk_text(text, chunk_size=800, overlap=100):
                self.add_text(chunk, source=fname)
        print(f"RAG index built with {len(self.records)} chunks.")

    def retrieve(self, query: str, top_k: int = 4) -> List[Dict[str, Any]]:
        if not self.records:
            return []
        q_emb = self.model.encode(query, convert_to_numpy=True)
        emb_matrix = np.vstack([r["emb"] for r in self.records])
        dots = emb_matrix @ q_emb
        qnorm = np.linalg.norm(q_emb) + 1e-12
        norms = np.linalg.norm(emb_matrix, axis=1) * qnorm + 1e-12
        sims = dots / norms
        top_idx = np.argsort(-sims)[:top_k]
        results = []
        for i in top_idx:
            results.append({"text": self.records[i]["text"], "source": self.records[i]["source"], "score": float(sims[i])})
        return results

def rag_suggest_clauses(question: str, top_k: int = 3) -> str:
    ensure_models()
    contexts = RAG_INDEX.retrieve(question, top_k=top_k)
    context_text = "\n\n".join([f"[source:{c['source']}] {c['text']}" for c in contexts])

    prompt = (
        "You are an ADGM legal compliance assistant. Use the following official ADGM reference contexts to answer accurately.\n\n"
        f"CONTEXTS:\n{context_text}\n\nQUESTION: {question}\n\nAnswer concisely:"
    )
    try:
        response = genai.chat.completions.create(
            model="models/chat-bison-001",
            messages=[
                {"author": "system", "content": "You are a helpful legal assistant."},
                {"author": "user", "content": prompt},
            ],
            temperature=0,
            max_tokens=300,
        )
        return response.candidates[0].content.strip()
    except Exception as e:
        print("Gemini API error:", e)
        return "Error generating suggestion."

RAG_INDEX: SimpleRAGIndex = None

def ensure_models():
    global RAG_INDEX
    if RAG_INDEX is None:
        RAG_INDEX = SimpleRAGIndex(EMBED_MODEL_NAME)
        RAG_INDEX.build_from_folder(REFS_DIR)

def classify_document(text: str) -> List[str]:
    found = []
    ln = (text or "").lower()
    for name, kws in DOCUMENT_KEYWORDS.items():
        if any(k in ln for k in kws):
            found.append(name)
    return found or ["Unknown"]

def detect_issues_in_text(text: str) -> List[Dict[str, Any]]:
    issues = []
    ln = (text or "").lower()
    for bad in JURISDICTION_KEYWORDS_WRONG:
        if bad in ln:
            issues.append({"section": "Jurisdiction", "issue": f"Incorrect jurisdiction reference '{bad}'", "severity": "High", "suggestion": "Replace with 'ADGM Courts'."})
            break
    if not any(k in ln for k in SIGNATURE_KEYWORDS):
        issues.append({"section": "Execution/Signatory", "issue": "Missing signature block", "severity": "High", "suggestion": "Add signature block with name/title/date."})
    for term in AMBIGUOUS_TERMS:
        if term in ln:
            issues.append({"section": "Language", "issue": f"Ambiguous term '{term.strip()}' used", "severity": "Medium", "suggestion": "Use binding/clear language."})
            break
    return issues

def insert_comments_and_highlight(input_docx_path: str, issues: List[Dict[str,Any]], out_path: str):
    doc = Document(input_docx_path)
    for issue in issues:
        phrase = issue.get("issue", "").split(":")[0].strip()
        highlighted = False
        for p in doc.paragraphs:
            if phrase and phrase.lower() in p.text.lower():
                run = p.add_run(f"  [REVIEWER NOTE: {issue['issue']}]")
                try:
                    run.font.highlight_color = WD_COLOR_INDEX.YELLOW
                except Exception:
                    pass
                highlighted = True
                break
        if not highlighted:
            p = doc.add_paragraph()
            run = p.add_run(f"[REVIEW] {issue['section']}: {issue['issue']} -- Suggestion: {issue.get('suggestion','')}")
            run.bold = True
    doc.save(out_path)

def analyze_docx_files(file_paths: List[str]) -> Tuple[Dict[str,Any], str]:
    ensure_models()
    tmpdir = tempfile.mkdtemp(prefix="adgm_review_")
    reviewed_files = {}
    detected_types = []
    issues_all = []
    uploaded_count = 0

    for path in file_paths:
        if not path.lower().endswith(".docx"):
            continue
        if not zipfile.is_zipfile(path):
            print("Skipping invalid DOCX:", path)
            continue
        fname = os.path.basename(path)
        uploaded_count += 1
        text = extract_text_from_docx(path)
        types = classify_document(text)
        detected_types.extend(types)
        issues = detect_issues_in_text(text)

        for iss in issues:
            if iss.get("severity") == "High":
                try:
                    sugg = rag_suggest_clauses(f"Provide a compliant ADGM clause for: {iss['section']}", top_k=3)
                    iss["suggestion"] = sugg
                except Exception:
                    iss["suggestion"] = iss.get("suggestion", "")

        for iss in issues:
            issues_all.append({
                "document": fname,
                "section": iss.get("section", "General"),
                "issue": iss.get("issue"),
                "severity": iss.get("severity"),
                "suggestion": iss.get("suggestion")
            })

        out_path = os.path.join(tmpdir, f"reviewed_{fname}")
        try:
            insert_comments_and_highlight(path, issues, out_path)
        except Exception:
            shutil.copy(path, out_path)
        reviewed_files[fname] = out_path

    process = "Unknown"
    if any(t in ["Articles of Association", "Memorandum of Association"] for t in detected_types):
        process = "Company Incorporation"

    required_documents = len(CHECKLIST)
    missing_document = None
    if process == "Company Incorporation":
        present = set(detected_types)
        missing = [d for d in CHECKLIST if d not in present]
        if missing:
            missing_document = missing[0]
    else:
        missing = []

    output = {
        "process": process,
        "documents_uploaded": uploaded_count,
        "required_documents": required_documents,
        "missing_document": missing_document,
        "issues_found": issues_all
    }

    zip_path = os.path.join(tmpdir, "reviewed_docs.zip")
    with zipfile.ZipFile(zip_path, "w") as zf:
        for orig, p in reviewed_files.items():
            if os.path.exists(p):
                zf.write(p, arcname=os.path.basename(p))

    return output, zip_path

def gradio_analyze(input_files):
    if not input_files:
        return {"error": "No files uploaded"}, ""
    paths = []
    for f in input_files:
        if isinstance(f, str):
            paths.append(f)
        else:
            try:
                p = getattr(f, "name", None) or f[0]
                paths.append(p)
            except Exception:
                continue
    report, zip_path = analyze_docx_files(paths)
    return report, zip_path

with gr.Blocks(title="ADGM Corporate Agent - Docx + Gemini RAG") as demo:
    gr.Markdown("# ADGM Corporate Agent — DOCX-only Reviewer with Gemini RAG")

    with gr.Tab("Document Analysis"):
        file_input = gr.File(file_count="multiple", file_types=[".docx"], label="Upload DOCX files")
        analyze_btn = gr.Button("Analyze Documents")
        output_json = gr.JSON(label="Structured Output")
        download = gr.File(label="Download reviewed files (zip)")

        analyze_btn.click(fn=gradio_analyze, inputs=[file_input], outputs=[output_json, download])

    with gr.Tab("Notes"):
        gr.Markdown("This tool accepts only valid `.docx` files. It will skip corrupted/non-docx files.\n"
                    "Uses Gemini (Google PaLM API) for RAG-powered suggestions.\n"
                    "Place official ADGM reference docs in the `adgm_refs` folder.")

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://09868767a4c6495e63.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
